In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/My\ Drive/Colab\ Notebooks/SCH_Proposal

/content/drive/My Drive/Colab Notebooks/SCH_Proposal


## The ROUGE evaluation is done between a generated summary for a cited article and its abstract (both can be accessed using their cited ids from the respective folders)

In [ ]:
!pip3 install -q rouge

In [ ]:
import pandas as pd
import numpy as np
import os
from rouge import Rouge
import json
from collections import defaultdict
from pprint import pprint

## Evaluation against the ground truth summary

In [ ]:
def _compute_ROUGE(generated_summary, human_summary):  
  rouge = Rouge()
  
  scores = rouge.get_scores(generated_summary, human_summary)[0]
  
  rouge_1_f = scores['rouge-1']['f']
  rouge_2_f = scores['rouge-2']['f']
  rouge_l_f = scores['rouge-l']['f']

  rouge_1_f = rouge_1_f * 100
  rouge_2_f = rouge_2_f * 100
  rouge_l_f = rouge_l_f * 100

  return rouge_1_f, rouge_2_f, rouge_l_f

## Iterate through the directories housing the generated and human summaries and store into a container and do evaluation

In [ ]:
!pwd

/content/drive/MyDrive/Colab Notebooks/SCH_Proposal


### Call to the ROUGE evaluation method in main method

In [ ]:
def main():
  lst_modelName = ["T5"]
  lst_model_type = ["vanilla", "w_named_entities"]
  #lst_model_type = ["vanilla", "w_named_entity"]
  ROOT_SUMMARY_PATH = "NLMCXR_reports_ecgen_radiology-FINAL-SUMMARIES"
  inference_type = "vanilla_at_inference"
  #inference_type = "w_named_entities_at_inference"
  output_file_name = f"NLMCXR_reports_ecgen_radiology_rouge_scores_wrt_input_article_{inference_type}.xlsx"
  evaluation_wrt_ground_truth = True    # changes based on what target to evaluate the generated to

  dict_rouge_scores = defaultdict(list)

  for modelName in lst_modelName:
    for model_type in lst_model_type:
      SUMMARY_PATH = f"{ROOT_SUMMARY_PATH}/{modelName}_{inference_type}"   # path changes with the type of input at inference time (w or wo named entities)
      #SUMMARY_PATH = f"pubmed-FINAL-SUMMARIES_w_named_entites_at_inference/{modelName}"   # path of summaries generated with named entities at inference
      input_filename_path = f"{SUMMARY_PATH}/NLMCXR_reports_ecgen_radiology-summaries-{model_type}.jsonl"

      output_results_path = f"FINAL_RESULTS/ROUGE"
      os.makedirs(output_results_path, exist_ok=True)
      

      rouge_1_f_sum, rouge_2_f_sum, rouge_l_f_sum = 0.0, 0.0, 0.0   # initialize all cumulative scores to zero
      total_no_summaries = 0
      with open(input_filename_path) as fp:
        for iter, line in enumerate(fp):
          if iter % 1000 == 0 and iter != 0:
            print("Iteration: ", iter)
          dict_data = json.loads(line)

          generated_summary = dict_data["abstractive_summary"]    # generated summary

          if evaluation_wrt_ground_truth:
            evaluation_target = dict_data["impression"]   # human-like summary as evaluation target
          else:
            evaluation_target = dict_data["finding"]   # source input article----to do evaluation wrt the input article

          # call to the ROUGE computing method
          try:
            rouge_1_f, rouge_2_f, rouge_l_f = _compute_ROUGE(generated_summary, evaluation_target)

            rouge_1_f_sum += rouge_1_f
            rouge_2_f_sum += rouge_2_f
            rouge_l_f_sum += rouge_l_f
            
            total_no_summaries += 1

          except:
            continue

      fp.close()

      avg_rouge_1_f = rouge_1_f_sum/float(total_no_summaries)
      avg_rouge_2_f = rouge_2_f_sum/float(total_no_summaries)
      avg_rouge_l_f = rouge_l_f_sum/float(total_no_summaries)

      dict_rouge_scores["training-config"].append(f"{modelName}-{model_type}")
      dict_rouge_scores["ROUGE-1"].append(avg_rouge_1_f)
      dict_rouge_scores["ROUGE-2"].append(avg_rouge_2_f)
      dict_rouge_scores["ROUGE-L"].append(avg_rouge_l_f)

      pprint(dict_rouge_scores)

  df_rouge_scores = pd.DataFrame(dict_rouge_scores)

  print(df_rouge_scores)

  df_rouge_scores.to_excel(f"{output_results_path}/{output_file_name}")  

In [ ]:
if __name__ == "__main__":
  main()

Iteration:  1000
Iteration:  2000
Iteration:  3000
defaultdict(<class 'list'>,
            {'ROUGE-1': [35.11325282585441],
             'ROUGE-2': [21.502578763151647],
             'ROUGE-L': [34.9207942666428],
             'training-config': ['T5-vanilla']})
Iteration:  1000
Iteration:  2000
Iteration:  3000
Iteration:  4000
Iteration:  5000
Iteration:  6000
Iteration:  7000
Iteration:  8000
Iteration:  9000
Iteration:  10000
defaultdict(<class 'list'>,
            {'ROUGE-1': [35.11325282585441, 10.956973232592574],
             'ROUGE-2': [21.502578763151647, 2.777513538689189],
             'ROUGE-L': [34.9207942666428, 10.780022823065202],
             'training-config': ['T5-vanilla', 'T5-w_named_entities']})
       training-config    ROUGE-1    ROUGE-2    ROUGE-L
0           T5-vanilla  35.113253  21.502579  34.920794
1  T5-w_named_entities  10.956973   2.777514  10.780023


## Entity-level Factual consistency

In [ ]:
!pip3 install -q scispacy
!pip3 install -q https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.3.0/en_core_sci_sm-0.3.0.tar.gz
!pip3 install -q https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

!python3 -m spacy download en

     |████████████████████████████████| 44 kB 1.8 MB/s 
     |████████████████████████████████| 13.5 MB 12.2 MB/s 
     |████████████████████████████████| 6.3 MB 2.5 MB/s 
     |████████████████████████████████| 71 kB 9.7 MB/s 
     |████████████████████████████████| 188 kB 68.0 MB/s 
     |████████████████████████████████| 10.1 MB 43.1 MB/s 
     |████████████████████████████████| 628 kB 61.6 MB/s 
     |████████████████████████████████| 451 kB 73.4 MB/s 
     |████████████████████████████████| 42 kB 1.8 MB/s 
     |████████████████████████████████| 33.1 MB 85 kB/s 
     |████████████████████████████████| 125.1 MB 33 kB/s 
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.7 MB 8.4 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ 

In [ ]:
!pip3 install -q jsonlines

In [ ]:
import spacy
import scispacy
import pandas as pd
import numpy as np
import pickle as pk
from pprint import pprint
import os

from spacy import displacy
import en_core_sci_sm
from scispacy.abbreviation import AbbreviationDetector
from scispacy.linking import EntityLinker
from collections import (OrderedDict,Counter, defaultdict)


from tqdm import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from pprint import pprint
import json
import jsonlines
from ast import literal_eval

/usr/local/lib/python3.7/dist-packages/spacy/util.py:718: UserWarning: [W094] Model 'en_core_sci_sm' (0.3.0) specifies an under-constrained spaCy version requirement: >=2.3.1. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.0.7,<3.1.0
  warnings.warn(warn_msg)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def _get_named_entities(input_text):
  nlp = spacy.load("en_ner_bc5cdr_md")
  doc = nlp(input_text)
  entities = []
  for ent in doc.ents:
    entities.append(ent.text)
  str_entities = " | ".join(entities)   # a string representation of list of entities with the pipe symbol as a separator
  
  return str_entities

### Extract named entities from the generated summaries

In [ ]:
def main():
  lst_modelName = ["T5"]
  #lst_model_type = ["vanilla"]
  lst_model_type = ["vanilla", "w_named_entities"]
  ROOT_SUMMARY_PATH = "NLMCXR_reports_ecgen_radiology-FINAL-SUMMARIES"
  inference_type = "vanilla_at_inference"
  #inference_type = "w_named_entities_at_inference"
  output_file_name = f"NLMCXR_reports_ecgen_radiology_rouge_scores_wrt_input_article_{inference_type}.xlsx"
  evaluation_wrt_ground_truth = True    # changes based on what target to evaluate the generated to

  for modelName in lst_modelName:
    for model_type in lst_model_type:
      SUMMARY_PATH = f"{ROOT_SUMMARY_PATH}/{modelName}_{inference_type}"
      input_filename_path = f"{SUMMARY_PATH}/NLMCXR_reports_ecgen_radiology-summaries-{model_type}.jsonl"

      OUTPUT_RESULTS_PATH = f"FINAL_RESULTS/factual_consistency"
      os.makedirs(OUTPUT_RESULTS_PATH, exist_ok=True)
      
      with open(input_filename_path) as fp:
        for iter, line in enumerate(fp):
          if iter % 2000 == 0:
            print("Iteration: ", iter)
          dict_data = json.loads(line)

          summary = dict_data["abstractive_summary"]
          summary_named_entities = _get_named_entities(summary)

          dict_1 = {"article_text" : dict_data['article_text'],
                    "article_abstract" : dict_data['article_abstract'],
                    "article_text_named_entities" : dict_data['article_text_named_entities'],
                    "article_abstract_named_entities" : dict_data['article_abstract_named_entities'],
                    "abstractive_summary" : dict_data["abstractive_summary"],
                    "abstractive_summary_named_entities" : summary_named_entities
                    }
              
          with jsonlines.open(f"{OUTPUT_RESULTS_PATH}/{output_file_name}", "a") as writer:
            writer.write(dict_1)
          writer.close()

      fp.close()


In [ ]:
if __name__ == "__main__":
  main()

## Entity-level Factual Consistency using F1 scores

In [ ]:
def main():
  lst_modelName = ["BART"]
  #lst_model_type = ["vanilla"]
  lst_model_type = ["vanilla", "w_named_entities"]
  ROOT_SUMMARY_PATH = "NLMCXR_reports_ecgen_radiology-FINAL-SUMMARIES"
  inference_type = "vanilla_at_inference"
  #inference_type = "w_named_entities_at_inference"
  output_file_name = f"NLMCXR_reports_ecgen_radiology_rouge_scores_wrt_input_article_{inference_type}.xlsx"
  evaluation_wrt_ground_truth = True    # changes based on what target to evaluate the generated to

  dict_entity_specificity = defaultdict(list)

  output_results_path = f"FINAL_RESULTS/ENTITY_SPECIFITY"
  os.makedirs(output_results_path, exist_ok=True)
  if evaluation_wrt_ground_truth:
    output_file_name = "OpenI_entity_specifity_scores_wrt_ground_truth_w_entity_at_inference.xlsx"
  else:
    output_file_name = "OpenI_entity_specifity_scores_wrt_source_article_w_entity_at_inference.xlsx"

  for modelName in lst_modelName:
    for model_type in lst_model_type:
      #SUMMARY_PATH = f"pubmed-FINAL-SUMMARIES/{modelName}"   # for vanilla input evaluation
      SUMMARY_PATH = f"{ROOT_SUMMARY_PATH}/{modelName}_{inference_type}"    # for input doc + named entity at inference time
      
      input_filename_path = f"{SUMMARY_PATH}/NLMCXR_reports_ecgen_radiology-summaries-{model_type}.jsonl"

      precision_total, recall_total = 0.0, 0.0   # precision and recall for entity specificity
      total_no_summaries = 0
      with open(input_filename_path) as fp:
        for iter, line in enumerate(fp):
          if iter % 2000 == 0:
            print("Iteration: ", iter)
          dict_data = json.loads(line)

          if evaluation_wrt_ground_truth:
            target_named_entities = dict_data['impression_entities'].split(' | ')    # ground truth named entities
          else:
            target_named_entities = dict_data['finding_entities'].split(' | ')   # source article named entities
          abstractive_summary_entities = dict_data['abstractive_summary_entities'].split(' | ')  # named entities in generated summary

          common_named_entities = list(set(target_named_entities) & set(abstractive_summary_entities))
          
          try:
            precision = len(common_named_entities) / float(len(abstractive_summary_entities))
            recall = len(common_named_entities) / float(len(target_named_entities))

            precision_total += precision
            recall_total += recall
            total_no_summaries += 1
          except:
            continue

      avg_precision = precision_total / float(total_no_summaries) * 100
      avg_recall = recall_total / float(total_no_summaries) * 100
      f1_score = (2 * avg_precision * avg_recall) / (avg_precision + avg_recall)

      dict_entity_specificity["training-config"].append(f"{modelName}-{model_type}")
      dict_entity_specificity["avg_precision"].append(avg_precision)
      dict_entity_specificity["avg_recall"].append(avg_recall)
      dict_entity_specificity["f1_score"].append(f1_score)
      
  df_entity_specifity = pd.DataFrame(dict_entity_specificity)

  print(df_entity_specifity)

  df_entity_specifity.to_excel(f"{output_results_path}/{output_file_name}")



In [ ]:
if __name__ == "__main__":
  main()

Iteration:  0
Iteration:  2000
Iteration:  0
Iteration:  2000
         training-config  avg_precision  avg_recall  f1_score
0           BART-vanilla       5.777032   29.678843  9.671494
1  BART-w_named_entities       0.754594    3.258058  1.225379


## MAUVE Evaluation

In [ ]:
!pip3 install -q mauve-text

     |████████████████████████████████| 8.5 MB 6.9 MB/s 


In [ ]:
!pip3 install -q torch>=1.10.0
!pip3 install -q git+https://github.com/huggingface/transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 7.2 MB/s 
     |████████████████████████████████| 895 kB 58.9 MB/s 
     |████████████████████████████████| 596 kB 75.6 MB/s 
     |████████████████████████████████| 61 kB 585 kB/s 


In [ ]:
import torch
import transformers
import mauve

In [ ]:
def main():
  lst_modelName = ["T5", "BART", "Pegasus"]
  lst_model_type = ["vanilla", "w_named_entities"]

  dict_entity_specificity = defaultdict(list)

  for modelName in lst_modelName:
    for model_type in lst_model_type:
      SUMMARY_PATH = f"pubmed-FINAL-SUMMARIES/{modelName}"
      input_filename_path = f"pubmed_FINAL-SUMMARIES-w-named_entities/{modelName}/pubmed-summaries-{model_type}.jsonl"

      lst_generated_summary = []
      lst_ground_truth_summary = []

      with open(input_filename_path) as fp:
        for iter, line in enumerate(fp):
          if iter % 2000 == 0:
            print("Iteration: ", iter)
          dict_data = json.loads(line)

          lst_generated_summary.append(dict_data["abstractive_summary"])    # generated summary
          lst_ground_truth_summary.append(dict_data["article_abstract"])   # human-like summary

      fp.close()
      
      lst_generated_summary = lst_generated_summary[:100]
      lst_ground_truth_summary = lst_ground_truth_summary[:100]

      out = mauve.compute_mauve(p_text=lst_ground_truth_summary, q_text=lst_generated_summary, verbose=False)

      mauve_score = out.mauve  
            
      dict_entity_specificity["training-config"].append(f"{modelName}-{model_type}")
      dict_entity_specificity["avg_mauve"].append(mauve_score)
      
      
  df_mauve_scores = pd.DataFrame(dict_entity_specificity)

  print(df_mauve_scores)

  df_mauve_scores.to_excel("pubmed_MAUVE_scores.xlsx")



In [ ]:
if __name__ == "__main__":
  main()